In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import yfinance as yf
import seaborn as sns
sns.set(style='darkgrid')

import sys
sys.path.append('../src') 

from utils import (stock_trading_strategy,
                   stock_trading_strategy_backwards,
                   stock_trading_strategy_finer,
                   plot_trading_strategy,
                   plot_trading_strategy_finer,
                   download_stock_df,
                   download_stock_df_full,
                   user_function,
                   plot_candlestick,
                  )

In [49]:
def calculate_investment_return(stock_prices, purchase_interval, investment_amount, start_date, end_date):
    # Filter stock prices between start and end date
    stock_prices = stock_prices[stock_prices['date'].between(start_date,end_date)]
    

    # Initialize variables
    total_investment = 0
    num_shares = 0
    daily_investment_value = list()
    daily_investment_amt = list()    
    last_purchase_date = None

    # Iterate over stock prices
    for idx, row in stock_prices.iterrows():
        # Check if it's time to make a purchase
        if last_purchase_date is None or (date - last_purchase_date).days >= purchase_interval:
            # Calculate number of shares to purchase
            num_shares += investment_amount / row['price']
            total_investment += investment_amount      
            last_purchase_date = row['date']
        else:
            num_shares += 0
            total_investment += 0
        
        # Calculate daily investment value
        daily_investment_value.append(num_shares * row['price'])
        daily_investment_amt.append(total_investment)

    df_strategy = stock_prices.copy()
    df_strategy['investment_value'] = daily_investment_value
    df_strategy['investment_amt_sofar'] = daily_investment_amt
    df_strategy['revenue'] = df_strategy['investment_value'] - df_strategy['investment_amt_sofar']
    df_strategy['revenue%'] = df_strategy['revenue'] / df_strategy['investment_amt_sofar']
    return df_strategy

# Example usage
# Assume stock_prices is a pandas dataframe with 'Date' as index and 'Price' as column
# Replace this with your actual data

purchase_interval = 7  # Weekly purchase
investment_amount = 1000  # $1000 per purchase
start_date = '2024-01-01'
end_date = '2024-01-31'

purchase_interval = 7  # Weekly purchase
investment_amount = 1000  # $1000 per purchase
start_date = '2024-01-01'
end_date = '2024-01-31'
stock_prices = pd.DataFrame({
    'date': pd.date_range(start='2024-01-01', periods=7, freq='D'),
    'price': [100, 110, 105, 120, 115, 125, 130],
})

stock_strategy =  calculate_investment_return(stock_prices, purchase_interval, investment_amount, start_date, end_date)
stock_strategy

,date,price,investment_value,investment_amt_sofar,revenue,revenue%
0,2024-01-01,100,1000.0,1000,0.0,0.00
1,2024-01-02,110,1100.0,1000,100.0,0.10
2,2024-01-03,105,1050.0,1000,50.0,0.05
3,2024-01-04,120,1200.0,1000,200.0,0.20
4,2024-01-05,115,1150.0,1000,150.0,0.15
5,2024-01-06,125,1250.0,1000,250.0,0.25
6,2024-01-07,130,1300.0,1000,300.0,0.30


In [59]:
df_brk = download_stock_df_full('brk-b')
stock_strategy =  calculate_investment_return(df_brk[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date='2022-01-01', 
                                              end_date=datetime.today().strftime('%Y-%m-%d'))
print(stock_strategy['revenue%'].min())
print(stock_strategy.tail(1)['revenue%'])

stock_strategy

[*********************100%%**********************]  1 of 1 completed
-0.16393585150568885
1548    0.297445
Name: revenue%, dtype: float64


,date,price,investment_value,investment_amt_sofar,revenue,revenue%
1008,2022-01-03,300.790009,500.000000,500,0.000000,0.000000
1009,2022-01-04,308.529999,1012.866103,1000,12.866103,0.012866
1010,2022-01-05,309.920013,1517.429350,1500,17.429350,0.011620
1011,2022-01-06,313.220001,2033.586739,2000,33.586739,0.016793
1012,2022-01-07,319.779999,2576.177647,2500,76.177647,0.030471
...,...,...,...,...,...,...
1544,2024-02-22,415.160004,328044.947572,251000,77044.947572,0.306952
1545,2024-02-23,417.220001,329672.685758,251000,78672.685758,0.313437
1546,2024-02-26,409.140015,323288.162325,251000,72288.162325,0.288001
1547,2024-02-27,408.910004,323106.415670,251000,72106.415670,0.287277


In [60]:
df_voo = download_stock_df_full('voo')
stock_strategy =  calculate_investment_return(df_voo[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date='2022-01-01', 
                                              end_date=datetime.today().strftime('%Y-%m-%d'))
print(stock_strategy['revenue%'].min())
print(stock_strategy.tail(1)['revenue%'])
stock_strategy

[*********************100%%**********************]  1 of 1 completed
-0.1470526545438937
1548    0.215341
Name: revenue%, dtype: float64


,date,price,investment_value,investment_amt_sofar,revenue,revenue%
1008,2022-01-03,439.250000,500.000000,500,0.000000,0.000000
1009,2022-01-04,439.070007,999.795114,1000,-0.204886,-0.000205
1010,2022-01-05,430.660004,1480.644909,1500,-19.355091,-0.012903
1011,2022-01-06,430.079987,1978.650762,2000,-21.349238,-0.010675
1012,2022-01-07,428.589996,2471.795827,2500,-28.204173,-0.011282
...,...,...,...,...,...,...
1544,2024-02-22,466.570007,305942.493203,251000,54942.493203,0.218894
1545,2024-02-23,466.779999,306080.190244,251000,55080.190244,0.219443
1546,2024-02-26,465.070007,304958.902890,251000,53958.902890,0.214976
1547,2024-02-27,465.929993,305522.818397,251000,54522.818397,0.217222


In [61]:
df_google= download_stock_df_full('GOOG')
stock_strategy =  calculate_investment_return(df_google[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date='2022-01-01', 
                                              end_date=datetime.today().strftime('%Y-%m-%d'))
print(stock_strategy['revenue%'].min())
print(stock_strategy.tail(1)['revenue%'])
stock_strategy

[*********************100%%**********************]  1 of 1 completed
-0.28890387020362757
1548    0.194191
Name: revenue%, dtype: float64


,date,price,investment_value,investment_amt_sofar,revenue,revenue%
1008,2022-01-03,145.074493,500.000000,500,0.000000,0.000000
1009,2022-01-04,144.416504,997.732236,1000,-2.267764,-0.002268
1010,2022-01-05,137.653503,1451.008604,1500,-48.991396,-0.032661
1011,2022-01-06,137.550995,1949.928059,2000,-50.071941,-0.025036
1012,2022-01-07,137.004501,2442.180946,2500,-57.819054,-0.023128
...,...,...,...,...,...,...
1544,2024-02-22,145.320007,316950.458903,251000,65950.458903,0.262751
1545,2024-02-23,145.289993,316884.996732,251000,65884.996732,0.262490
1546,2024-02-26,138.750000,302620.932812,251000,51620.932812,0.205661
1547,2024-02-27,140.100006,305565.366011,251000,54565.366011,0.217392


In [62]:
df_qqq =  download_stock_df_full('qqq')
stock_strategy =  calculate_investment_return(df_qqq[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date='2022-01-01', 
                                              end_date=datetime.today().strftime('%Y-%m-%d'))
print(stock_strategy['revenue%'].min())
print(stock_strategy.tail(1)['revenue%'])
stock_strategy

[*********************100%%**********************]  1 of 1 completed
-0.18730736363198602
1548    0.344876
Name: revenue%, dtype: float64


,date,price,investment_value,investment_amt_sofar,revenue,revenue%
1008,2022-01-03,401.679993,500.000000,500,5.684342e-14,1.136868e-16
1009,2022-01-04,396.470001,993.514749,1000,-6.485251e+00,-6.485251e-03
1010,2022-01-05,384.290009,1462.992887,1500,-3.700711e+01,-2.467141e-02
1011,2022-01-06,384.019989,1961.964922,2000,-3.803508e+01,-1.901754e-02
1012,2022-01-07,379.859985,2440.711390,2500,-5.928861e+01,-2.371544e-02
...,...,...,...,...,...,...
1544,2024-02-22,438.070007,339735.475076,251000,8.873548e+04,3.535278e-01
1545,2024-02-23,436.779999,338735.037569,251000,8.773504e+04,3.495420e-01
1546,2024-02-26,436.549988,338556.657652,251000,8.755666e+04,3.488313e-01
1547,2024-02-27,437.600006,339370.976056,251000,8.837098e+04,3.520756e-01


In [63]:
df_tqqq =  download_stock_df_full('tqqq')
stock_strategy =  calculate_investment_return(df_tqqq[['date','close']].rename(columns={'close':'price'}),
                                              purchase_interval=7, 
                                              investment_amount=500, 
                                              start_date='2022-01-01', 
                                              end_date=datetime.today().strftime('%Y-%m-%d'))
print(stock_strategy['revenue%'].min())
print(stock_strategy.tail(1)['revenue%'])
stock_strategy

[*********************100%%**********************]  1 of 1 completed
-0.49985202676836377
1548    0.909557
Name: revenue%, dtype: float64


,date,price,investment_value,investment_amt_sofar,revenue,revenue%
1008,2022-01-03,85.565002,500.000000,500,0.000000,0.000000
1009,2022-01-04,82.239998,980.570300,1000,-19.429700,-0.019430
1010,2022-01-05,74.675003,1390.370769,1500,-109.629231,-0.073086
1011,2022-01-06,74.470001,1886.553849,2000,-113.446151,-0.056723
1012,2022-01-07,72.035004,2324.867882,2500,-175.132118,-0.070053
...,...,...,...,...,...,...
1544,2024-02-22,60.169998,489301.053496,251000,238301.053496,0.949407
1545,2024-02-23,59.619999,484828.472238,251000,233828.472238,0.931588
1546,2024-02-26,59.540001,484177.930193,251000,233177.930193,0.928996
1547,2024-02-27,59.910000,487186.753073,251000,236186.753073,0.940983
